LSTM for sentiment analysis on 1.6 dataset

In [61]:
import os
os.chdir("/Users/imacair/Desktop/Products3/")

In [62]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

In [63]:
data = pd.read_csv('trainingandtestdata/training.1600000.processed.noemoticon.csv',
                   names =['sentiment', 'id', 'date', 'query', 'user', 'text'], encoding='latin1',delimiter=',')
data.text = data.text.str.encode('ascii','replace')

In [64]:
data= data.sample(n=10000)

In [65]:
data = data[['text','sentiment']]

In [66]:
#data2 = pd.read_csv('Final_Manual_0805.csv', encoding='latin1',delimiter=',')
#data2 = data2[['message','sentiment']]

In [67]:
print(data[ data['sentiment'] == 0].size)
print(data[ data['sentiment'] == 4].size)

10250
9750


In [68]:
data.size

20000

In [69]:
max_fatures = 2000
tokenizer = Tokenizer(nb_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)

In [70]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1], dropout=0.2))
model.add(LSTM(lstm_out, dropout_U=0.2, dropout_W=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

/Users/imacair/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
/Users/imacair/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:6: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(196, dropout=0.2, recurrent_dropout=0.2)`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 32, 128)           256000    
_________________________________________________________________
lstm_2 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 394       
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


In [71]:
Y = pd.get_dummies(data['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

((6700, 32), (6700, 2))
((3300, 32), (3300, 2))


In [72]:
batch_size = 32
model.fit(X_train, Y_train, nb_epoch = 7, batch_size=batch_size, verbose = 2)


Epoch 1/7
51s - loss: 0.6032 - acc: 0.6628
Epoch 2/7
49s - loss: 0.4713 - acc: 0.7770
Epoch 3/7
49s - loss: 0.4152 - acc: 0.8115
Epoch 4/7
49s - loss: 0.3574 - acc: 0.8428
Epoch 5/7
48s - loss: 0.3074 - acc: 0.8670
Epoch 6/7
49s - loss: 0.2633 - acc: 0.8845
Epoch 7/7
48s - loss: 0.2224 - acc: 0.9046


In [73]:
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

score: 0.84
acc: 0.71


In [42]:
out=model.predict(X_validate)

In [45]:
Y_validate

array([[ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       ..., 
       [ 0.,  1.],
       [ 1.,  0.],
       [ 0.,  1.]])

In [44]:
print(out)

[[ 0.88032293  0.11967707]
 [ 0.88329941  0.1167006 ]
 [ 0.98877501  0.01122495]
 ..., 
 [ 0.0075256   0.99247438]
 [ 0.07892516  0.92107481]
 [ 0.00388306  0.996117  ]]


In [57]:
for x in range(len(out)):
    if out[x][0]> out[x][1]:
        out[x][0] = 1
        out[x][1] = 0
    else:
        out[x][0] = 0
        out[x][1] = 1

In [60]:
print(classification_report(Y_validate,out))

             precision    recall  f1-score   support

          0       0.69      0.71      0.70       752
          1       0.70      0.68      0.69       748

avg / total       0.69      0.69      0.69      1500



In [21]:
from sklearn.metrics import classification_report

Link https://www.kaggle.com/ngyptr/lstm-sentiment-analysis-keras